In [1]:
# Ankit Kumar Sahoo
# 19IM10006

from numpy.random import randint
from numpy.random import rand

D = 60
K = 0.15
rt = 0.01
#format of the data 
#[Loan Size -> L, interest -> rl,rating, loss -> lambda]
#Customers details provided in the question 
customers = [[10, 0.021, 'AAA', 0.0002],
             [25, 0.022, 'BB', 0.0058],
             [4, 0.021, 'A', 0.001],
             [11, 0.027, 'AA', 0.0003],
             [18, 0.025, 'BBB', 0.0024],
             [3, 0.026, 'AAA', 0.0002],
             [17, 0.023, 'BB', 0.0058],
             [15, 0.021, 'AAA', 0.0002],
             [9, 0.028, 'A', 0.001],
             [10, 0.022, 'A', 0.001]
            ]

In [2]:
# objective function (Basically fitness function)
# rl * L  - lambda +  rt*Transactional cost  - 0.009*D - lambda
def fitness(x):
  sum  = -0.009*D
  for i in range(len(x)):
    if x[i] == 1:
      transaction_cost = (1-K)*D - customers[i][0]
      sum = sum + customers[i][1]*customers[i][0] - 2*customers[i][3] + 0.01 * transaction_cost
  return (sum)

In [3]:
#Genetic Validation
def validation(x):
  sum = 0
  for i in range(len(x)):
    if x[i] == 1:
      sum = sum + customers[i][0] 
  return (sum)

In [4]:
# Roulette wheel selection method
# Generating the cummulative prob of selecting the chromosome based on fitness
# Selecting the individual using rando numbers 
def roulette_select(population, num):
    fitness_arr = []
    for i in population:
      fitness_arr.append(fitness(i))

    total_fitness = float(sum(fitness_arr))

    cummulative_fitness = []
    for i in fitness_arr:
      cummulative_fitness.append(i/total_fitness)
    
    prob = cummulative_fitness
    for i in range(1,len(cummulative_fitness)):
      prob[i] += prob[i-1]

    new_population = []
    for n in range(num):
        r = rand()
        for (i, individual) in enumerate(population):
            if r <= prob[i]:
                new_population.append(individual)
                break
    return new_population

In [5]:
# Single point crossover where R_CROSS is crossover probability
# Implemented genetic validation as well so that newly generated chromosomes follows constraints
def crossover(p1, p2, r_cross):
  c1, c2 = p1.copy(), p2.copy() 
  if rand() < r_cross:
    pt = randint(1, len(p1)-2)
    c1 = p1[:pt] + p2[pt:]
    c2 = p2[:pt] + p1[pt:]
  if validation(c1) > (1-K)*D :
    c1 = p1
  if validation(c2) > (1-K)*D :
    c2 = p2
  return [c1, c2]

In [6]:
# Implemented mutation using mutation prob
# Implemented mutation validation as well
def mutation(bitstr, r_mut):
  for i in range(len(bitstr)):
    if rand() < r_mut:
      bitstr[i] = 1 - bitstr[i]   
      if validation(bitstr)>((1-K)*D) :
        bitstr[i] = 1 - bitstr[i]

In [7]:
# Genetic Algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    pop = []
    i = 1
    while i <= n_pop:
        curr = randint(0, 2, n_bits).tolist()
        if validation(curr)<= (1-K)*D:
          pop.append(curr)
          i = i + 1
    best,best_eval = 0, objective(pop[0])
    for gen in range(n_iter):
        scores = [objective(c) for c in pop]
        for i in range(n_pop):
          if scores[i] > best_eval:
            best, best_eval = pop[i], scores[i]
        print("Generation %d: New Best fitness(%s) = %.3f" % (gen+1,  pop[i], scores[i]))
        selected = roulette_select(pop,n_pop)
        children = list()
        for i in range(0, n_pop, 2):
          p1, p2 = selected[i], selected[i+1]
          for c in crossover(p1, p2, r_cross):
            mutation(c, r_mut)
            children.append(c)
        pop = children
    return [best, best_eval]

In [8]:
# total iterations
n_iter = 100
# bits
n_bits = 10
# population size
n_pop = 10
# crossover rate
r_cross = 0.8
# mutation rate
r_mut = 0.05
best, score = genetic_algorithm(fitness, n_bits, n_iter, n_pop, r_cross, r_mut)
print('GENETIC ALGORITHM')
print('fitness(%s) = %f' % (best, score))

Generation 1: New Best fitness([1, 0, 1, 0, 1, 1, 0, 0, 0, 0]) = 1.964
Generation 2: New Best fitness([1, 0, 0, 1, 0, 0, 0, 0, 1, 0]) = 1.446
Generation 3: New Best fitness([0, 0, 0, 0, 0, 1, 1, 1, 1, 0]) = 2.082
Generation 4: New Best fitness([0, 0, 1, 0, 1, 1, 1, 0, 0, 0]) = 2.064
Generation 5: New Best fitness([1, 0, 1, 0, 0, 1, 1, 0, 0, 1]) = 2.537
Generation 6: New Best fitness([0, 0, 1, 0, 0, 1, 1, 1, 0, 0]) = 1.964
Generation 7: New Best fitness([1, 0, 1, 0, 0, 1, 1, 0, 1, 0]) = 2.579
Generation 8: New Best fitness([1, 0, 1, 0, 0, 1, 1, 0, 1, 0]) = 2.579
Generation 9: New Best fitness([1, 0, 1, 0, 0, 1, 1, 0, 1, 0]) = 2.579
Generation 10: New Best fitness([1, 0, 1, 0, 1, 0, 1, 0, 0, 0]) = 2.126
Generation 11: New Best fitness([1, 0, 1, 0, 0, 0, 1, 0, 0, 0]) = 1.351
Generation 12: New Best fitness([1, 0, 1, 0, 0, 1, 1, 0, 1, 0]) = 2.579
Generation 13: New Best fitness([0, 0, 1, 0, 1, 0, 1, 0, 0, 0]) = 1.507
Generation 14: New Best fitness([1, 0, 1, 0, 0, 0, 0, 0, 1, 0]) = 1.302
G

In [9]:
print(best)

[1, 0, 1, 1, 0, 1, 0, 0, 1, 1]


In [10]:
validation(best)

47

In [11]:
print(score)

3.1836
